In [3]:
import numpy as np
import tensorflow as tf
import keras
import os
import sys
from tqdm import tqdm
import time
import keras.backend as K

2022-03-29 11:21:10.059206: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
sys.path.append('..')

In [112]:
import nets
from Data import datagen
import importlib 
import resnet1D_Ahmed
importlib.reload(datagen)  # Python 3.4+

<module 'Data.datagen' from '/home/keondopark/sleep/JupyterNotebooks/../Data/datagen.py'>

In [6]:
np.random.seed(1)

In [7]:
PROCESSED_DATA_PATH = os.path.join('/home','aiot','data','origin_npy')
save_signals_path_SC = os.path.join(PROCESSED_DATA_PATH,'signals_SC_filtered')
save_annotations_path_SC = os.path.join(PROCESSED_DATA_PATH,'annotations_SC')
save_signals_path_ST = os.path.join(PROCESSED_DATA_PATH,'signals_ST_filtered')
save_annotations_path_ST = os.path.join(PROCESSED_DATA_PATH,'annotations_ST')

In [8]:
def match_annotations_npy(dirname, filepath):
    filename = os.path.basename(filepath)
    search_filename = filename.split('-')[0][:-2]
    file_list = os.listdir(dirname)
    filenames = [file for file in file_list if search_filename in file if file.endswith('.npy')]
    return filenames

In [9]:
dim_HT1D = (3000,1)
n_classes=6
epochs = 50
bs = 64
BASE_LEARNING_RATE = 1e-3
data_ratio = 0.25
PREV_CNT = 10
list_files_SC = [os.path.join(save_signals_path_SC, f) for f in os.listdir(save_signals_path_SC) if f.endswith('.npy')]
list_files_ST = [os.path.join(save_signals_path_ST, f) for f in os.listdir(save_signals_path_ST) if f.endswith('.npy')]

In [130]:
train_test_split = 0.7
split_cnt_SC = int(train_test_split * len(list_files_SC))
split_cnt_ST = int(train_test_split * len(list_files_ST))

list_files_train = []
list_files_test = []

list_ann_files_train = []
list_ann_files_test = []


list_files_SC_train = np.random.choice(list_files_SC[:split_cnt_SC], int(0.25 * split_cnt_SC), replace=False)
list_files_train += list_files_SC_train.tolist()
for f in list_files_SC_train:
    ann_file = match_annotations_npy(save_annotations_path_SC, f)
    list_ann_files_train.append(os.path.join(save_annotations_path_SC, ann_file[0]))

list_files_test += list_files_SC[split_cnt_SC:]

for f in list_files_SC[split_cnt_SC:]:
    ann_file = match_annotations_npy(save_annotations_path_SC, f)
    list_ann_files_test.append(os.path.join(save_annotations_path_SC, ann_file[0]))


list_files_ST_train = np.random.choice(list_files_ST[:split_cnt_ST], int(0.25 * split_cnt_ST), replace=False)
list_files_train += list_files_ST_train.tolist()
for f in list_files_ST_train:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files_train.append(os.path.join(save_annotations_path_ST, ann_file[0]))

list_files_test += list_files_ST[split_cnt_ST:]
for f in list_files_ST[split_cnt_ST:]:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files_test.append(os.path.join(save_annotations_path_ST, ann_file[0]))

list_files_train2 = list_files_train[:1]
list_ann_files_train2 = list_ann_files_train[:1]
    
    
train_generator = datagen.DataGenerator2(list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=True, prev_cnt=PREV_CNT)
test_generator = datagen.DataGenerator2(list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False, prev_cnt=PREV_CNT)

In [129]:
x, y, bi = next(iter(train_generator))

In [115]:
train_generator_o = datagen.DataGenerator(list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=True)

In [116]:
x, y, bi = next(iter(train_generator))

In [117]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [118]:
len(train_generator)

1446

In [133]:
cnt = 0
for x, y, bi in train_generator:
    cnt += 1
    if cnt < 100:
        print(cnt, y)
    else:
        break

1 [2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 0 0 1 1 1 1 2 2 1 0 0 0 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2]
2 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
3 [4 4 4 4 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 2 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
4 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 1 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
5 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
6 [0 1 1 0 0 0 1 0 0 0 0 1 1 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
7 [2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 2 2 2 2 2 2 2 2 2 2 2 4 4 4]
8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

76 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
77 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
78 [4 4 4 4 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
79 [2 2 2 2 2 2 2 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4]
80 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
81 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
82 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
83 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [120]:
ann = np.load(list_ann_files_train2[0])

In [121]:
ann[-10:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [122]:
cnt = 0
for x, y in train_generator_o:
    cnt += 1
    if cnt < 100:
        print(cnt, y)
    else:
        break

1 [0 2 0 0 1 0 0 0 0 1 0 0 4 1 1 2 1 0 0 0 0 1 2 0 0 2 0 0 0 2 0 0 0 1 0 0 0
 2 0 0 1 0 2 0 0 0 1 1 1 0 2 4 0 0 0 0 0 1 0 0 0 0 4 0]
2 [2 0 1 0 0 0 0 0 1 0 0 2 2 0 2 2 2 0 1 4 2 2 0 0 2 2 0 0 2 2 0 2 0 0 1 0 2
 2 1 1 0 0 4 1 1 1 4 0 2 1 0 0 0 0 2 1 0 0 2 0 0 1 1 0]
3 [2 4 0 0 1 2 0 0 4 0 2 1 0 2 0 2 1 1 0 2 0 1 0 0 0 0 0 2 0 1 0 2 4 1 0 1 0
 0 1 0 2 2 1 0 2 0 0 0 0 0 2 0 1 0 1 0 1 2 0 2 2 1 0 2]
4 [2 0 2 0 1 0 0 0 0 4 0 0 0 1 0 0 2 0 0 1 1 1 0 0 2 0 2 4 4 1 0 0 1 0 1 0 0
 4 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 2 1 1 2 0 2 1 0 0 0 0]
5 [1 0 0 1 1 0 0 0 1 0 0 2 0 0 1 0 1 1 2 0 0 2 0 1 0 0 0 2 2 2 0 0 0 0 2 1 2
 0 2 0 4 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0 2 0 0 0]
6 [0 0 0 0 0 0 0 0 0 2 1 0 0 0 4 0 0 0 0 0 1 0 0 1 0 0 0 2 0 0 0 1 0 0 0 2 0
 1 1 0 0 0 0 0 0 2 0 0 0 2 0 0 2 0 1 4 1 0 0 1 0 0 0 0]
7 [1 0 0 4 2 0 2 0 0 2 0 2 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 2 0 0 0 1 2 1 0 0
 4 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 2 0 2 0 2]
8 [0 0 0 0 0 0 0 2 4 1 0 0 1 0 0 0 1 0 0 0 2 1 4 0 4 2 1 0 2 0 4 4 0 

63 [0 2 0 2 2 0 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0
 0 0 0 0 1 2 0 0 0 0 0 0 0 2 0 2 2 2 0 0 0 1 0 0 0 0 2]
64 [1 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 0 0 0 0 1 0 2 2 0 4 2 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0]
65 [2 0 2 0 0 0 0 0 0 1 0 0 0 2 1 0 0 0 0 0 0 0 0 0 2 0 2 4 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 2 0 1 0 0 0 0 0 0 0 0 0 1 0 0 2 0 4 0 2 2 0]
66 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 4 0 2 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 2 2 1 0 0 2 0 0 4 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0 0]
67 [0 0 0 2 0 0 0 0 0 0 2 1 0 0 4 0 2 0 2 2 2 0 0 0 0 2 0 0 4 1 2 0 0 0 0 0 0
 0 0 0 0 2 0 1 0 2 0 0 0 0 0 0 0 0 2 0 0 2 1 0 0 2 0 0]
68 [2 0 0 0 2 0 0 4 0 0 0 4 2 4 2 0 0 0 0 0 0 2 0 0 0 0 4 0 4 0 0 0 2 0 0 4 0
 2 2 1 0 0 0 0 0 0 1 0 0 0 2 0 0 0 0 0 2 2 0 0 2 1 1 0]
69 [0 4 0 0 0 2 0 2 0 1 0 0 4 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 2 2 0 0 0 0 2 0 0 2 0 1 0 0 2 2 1 0 0 0 0 0 0]
70 [0 1 0 2 4 0 2 0 0 0 4 0 0 0 1 0 0 0 0 0 0 4 0 0 0 0 2 0 0 

In [131]:
from collections import defaultdict
cnt_class = defaultdict(int)
for x, y in train_generator_o:
    unique, counts = np.unique(y, return_counts=True)
    for i, cnt in zip(unique, counts):
        cnt_class[i] += cnt
cnt_class_np = np.zeros((n_classes,))
for i in range(n_classes):
    cnt_class_np[i] = cnt_class[i]
class_weight = 0.1 * np.ones((n_classes,))
class_weight[:n_classes-1] = sum(cnt_class_np[:n_classes-1])/(n_classes * cnt_class_np[:n_classes-1])

In [132]:
#array([49845.,  5164., 13992.,  2675.,  5113.,   139.])
cnt_class_np

array([49058.,  5067., 14266.,  3202.,  5844.,   126.])

In [127]:
np.sum(cnt_class_np)/64

1225.0625

In [125]:
# Calculate class weight
# Tested loss with class weight, but doesn't improve the accuracy

from collections import defaultdict
cnt_class = defaultdict(int)
for x, y, batch_idx in train_generator:
    y1 = y
    if batch_idx > 0:
        y1 = y[PREV_CNT:]
    unique, counts = np.unique(y1, return_counts=True)
    for i, cnt in zip(unique, counts):
        cnt_class[i] += cnt
cnt_class_np = np.zeros((n_classes,))
for i in range(n_classes):
    cnt_class_np[i] = cnt_class[i]
class_weight = 0.1 * np.ones((n_classes,))
class_weight[:n_classes-1] = sum(cnt_class_np[:n_classes-1])/(n_classes * cnt_class_np[:n_classes-1])


In [126]:
#array([49640.,  5117., 14360.,  3228.,  5909.,   150.])
cnt_class_np

array([49640.,  5117., 14360.,  3228.,  5909.,   150.])

In [150]:
len(list_ann_files_train), len(list_ann_files_test)

(32, 58)

In [151]:
list_ann_files_train

['/home/aiot/data/origin_npy/annotations_SC/SC4261FM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4661EJ-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4722EM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4641EP-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4091EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4742EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4021EH-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4451FY-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4532EV-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4042EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4571FV-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4502EM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4401EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4732EJ-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annot

In [152]:
model = nets.Conv1DASPP_prev()

In [153]:
x = np.random.random((64,3000,1))
x = tf.convert_to_tensor(x)
model(x)

<tf.Tensor: shape=(64, 6), dtype=float32, numpy=
array([[0.16973524, 0.16630745, 0.17377722, 0.16021399, 0.17130591,
        0.15866019],
       [0.16921692, 0.16681409, 0.17344151, 0.16024154, 0.1706118 ,
        0.15967411],
       [0.16902594, 0.16752543, 0.17219436, 0.16109498, 0.17112027,
        0.159039  ],
       [0.16920619, 0.1661851 , 0.17330651, 0.1603453 , 0.17140149,
        0.15955546],
       [0.16931668, 0.16737747, 0.17394885, 0.16013874, 0.17098126,
        0.158237  ],
       [0.16844678, 0.16701986, 0.17322499, 0.16077387, 0.17054735,
        0.15998715],
       [0.16878071, 0.16637264, 0.17452267, 0.1602637 , 0.16990496,
        0.1601553 ],
       [0.16927868, 0.1670752 , 0.17414981, 0.16002983, 0.1702515 ,
        0.15921493],
       [0.16942756, 0.16713938, 0.17369036, 0.16060594, 0.17013408,
        0.15900266],
       [0.16924617, 0.1672862 , 0.17300035, 0.16008957, 0.17148034,
        0.15889736],
       [0.16988957, 0.1674604 , 0.1722386 , 0.16028856, 0.170

In [159]:
model.summary()

Model: "conv1daspp_prev_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_block_79 (conv1d_bloc multiple                  320       
_________________________________________________________________
conv1d_block_80 (conv1d_bloc multiple                  10560     
_________________________________________________________________
conv1d_block_81 (conv1d_bloc multiple                  20800     
_________________________________________________________________
conv1d_block_82 (conv1d_bloc multiple                  41280     
_________________________________________________________________
conv1d_block_83 (conv1d_bloc multiple                  41280     
_________________________________________________________________
conv1d_block_84 (conv1d_bloc multiple                  41280     
_________________________________________________________________
conv1d_block_85 (conv1d_bloc multiple            

In [160]:
def get_current_lr(epoch):
    lr = BASE_LEARNING_RATE
    for _ in range(epoch // 10):
        lr *= 0.1
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    optimizer.learning_rate = lr

In [161]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        bs = y_pred.shape[0]
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        #loss = -K.sum(loss, -1)
        loss = -K.sum(loss) / bs
        return loss
    
    return loss

In [162]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
#loss_fn = weighted_categorical_crossentropy(weights=class_weight)

In [163]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './ckpt_Advanced_Conv1D', max_to_keep=1)

In [135]:
start_epoch = 0
#if manager.latest_checkpoint:
#    ckpt.restore(manager.latest_checkpoint)
#    start_epoch = ckpt.step.numpy()-1
best_test_acc = 0.0

In [136]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = loss_fn(y, y_pred)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))    
    return loss_value, y_pred

@tf.function
def test_step(x, y):
    y_pred = model(x, training=False)
    return y_pred


In [141]:
train_generator.batch_indexes[train_generator.file_indexes[0]]

array([11, 10, 16,  2, 27, 25, 29, 23,  8,  3, 20,  6, 46, 22, 24, 26, 33,
        0, 41, 19, 13,  4,  9, 12, 14, 28, 21, 42, 31, 35, 17, 18, 44, 40,
        5, 39,  7, 45, 30, 43,  1, 34, 36, 32, 38, 37, 15])

In [ ]:
for e in range(start_epoch, epochs):
    correct, total_cnt, total_loss = 0.0, 0.0, 0.0
    print('-'*20 + 'Epoch ' + str(e) + '-'*20)
    adjust_learning_rate(optimizer, e)
    start = time.time()
    for idx, (x, y) in enumerate(train_generator):   
        #print(x.shape)
        #loss, y_pred = train_step(x, y)
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            loss_value = loss_fn(y, y_pred)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))    
        loss = loss_value

        total_cnt += y_pred.shape[0]
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_loss += loss * y_pred.shape[0]
        if (idx + 1) % 10 == 0 or idx+1 == len(train_generator):
            print("[%d / %d] Training loss: %.6f, Training acc: %.3f"%
                  (idx+1, len(train_generator), total_loss / total_cnt, correct / total_cnt),end='\r', flush=True)
        
    print("")
    print("Training loss: %.6f, Training acc: %.3f"%(total_loss / total_cnt, correct / total_cnt))
    print("Training time: %.2f sec "%(time.time() - start))
    ckpt.step.assign_add(1)
    
    if e+1 >= 10 and (e+1) % 5 == 0:
        start = time.time()
        
        correct, total_cnt, total_loss = 0.0, 0.0, 0.0
        for idx, (x, y) in enumerate(test_generator):
            #y_pred = model(x, training=False)
            
            y_pred = test_step(x, y)
            y_pred_cls = tf.math.argmax(y_pred, axis=-1)
            correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
            total_cnt += y_pred.shape[0]
            y = tf.cast(y, dtype=tf.int32)
            y_onehot = tf.one_hot(y, depth=n_classes)
            total_loss += loss_fn(y, y_pred).numpy() * y_pred.shape[0]
            #total_loss += loss_fn(y_onehot, y_pred).numpy() * y_pred.shape[0]
                
            test_acc = correct / total_cnt
            test_loss = total_loss / total_cnt
            if (idx + 1) % 10 == 0 or idx+1 == len(test_generator):
                print("[%d / %d] test loss: %.6f, test accuracy: %.3f"%
                    (idx+1, len(test_generator), test_loss, test_acc),end='\r', flush=True)
            
        print("")
        print("test loss: %.6f, test acc: %.3f"%(test_loss, test_acc))
        print("Eval time: %.2f sec"%(time.time() - start))
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))  

--------------------Epoch 0--------------------
[1211 / 1211] Training loss: 1.572030, Training acc: 0.619
Training loss: 1.572030, Training acc: 0.619
Training time: 178.18 sec 
--------------------Epoch 1--------------------
[1211 / 1211] Training loss: 0.938574, Training acc: 0.687
Training loss: 0.938574, Training acc: 0.687
Training time: 178.11 sec 
--------------------Epoch 2--------------------
[1211 / 1211] Training loss: 0.957514, Training acc: 0.671
Training loss: 0.957514, Training acc: 0.671
Training time: 179.98 sec 
--------------------Epoch 3--------------------
[1211 / 1211] Training loss: 0.801773, Training acc: 0.719
Training loss: 0.801773, Training acc: 0.719
Training time: 185.13 sec 
--------------------Epoch 4--------------------
[1211 / 1211] Training loss: 0.730770, Training acc: 0.737
Training loss: 0.730770, Training acc: 0.737
Training time: 175.08 sec 
--------------------Epoch 5--------------------
[1211 / 1211] Training loss: 0.730808, Training acc: 0.74

In [ ]:
correct, total_cnt, total_loss = 0.0, 0.0, 0.0
confusion_matrix = np.zeros((n_classes,n_classes))
for idx, (x, y) in enumerate(test_generator):
    y_pred = model(x, training=False)
    y_pred_cls = tf.math.argmax(y_pred, axis=-1)
    correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
    total_cnt += y_pred.shape[0]
    y = tf.cast(y, dtype=tf.int32)    
    for i in range(n_classes):
        for j in range(n_classes):
            confusion_matrix[i,j] += np.sum((y_pred_cls.numpy()==i) * (y.numpy()==j))


In [ ]:
for i in range(n_classes):
    print_ln = ""
    for j in range(n_classes):
        print_ln += "%.3f "%(confusion_matrix[i,j] / np.sum(confusion_matrix[i]))
        #print_ln += "%d "%(confusion_matrix[i,j])
    print(print_ln)

In [ ]:
x = np.arange(10)
np.random.seed(1)
np.random.shuffle(x)

In [ ]:
x

In [ ]:
y = np.arange(20)
np.random.shuffle(y)

In [ ]:
y

In [ ]:
pc = np.array([[0,0,1],[1,0,0],[0,1,0], [3,4,5]])

In [ ]:
n = pc.shape[0]
x = np.tile(np.expand_dims(pc,1), [1,n,1])
y = np.empty((n,n,3))
y[:] = np.tile(np.expand_dims(pc,0), [n,1,1])
dist = np.sum((x - y) ** 2, axis=2) ** 0.5 # n by n matrix

In [ ]:
instances_list = []
thr = 2
for i in range(1, n):    
    included = False
    for inst_set in instances_list:
        if i in inst_set:
            included = True
            break
    if not included:
        close_pts = set()           
        q = [i]
        while q:
            j = q.pop()
            #if j in close_pts: continue
            new_pts = set(np.where(dist[j,:] < thr)[0])
            add_pts = new_pts - close_pts
            q += list(add_pts)
            close_pts = close_pts.union(add_pts)
        
        instances_list.append(close_pts)
centroids = []
for s in instances_list:
    cent = np.mean(pc[list(s),:], axis=0)
    centroids.append(cent)
    
    

In [ ]:
centroids, instances_list

In [ ]:
x = np.array([[[1,2,3,4],[5,6,7,8],[4,3,2,1]], [[1,2,3,4],[5,9,7,8],[4,3,2,1]]])

In [ ]:
x.shape

In [ ]:
x[:,:,0] == 5

In [ ]:
np.sum((x[:,:,0] == 5) * (x[:,:,1] == 6))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
a = np.random.random((2,3000))
b = np.ones(2)
c = 'abc'

In [ ]:
df = pd.DataFrame(a)
df['b'] = b
df['c'] = c

In [ ]:
df

In [ ]:
d = np.random.random((2,3000))
e = np.ones(2)*2
f = 'def'
df2 = pd.DataFrame(d)
df2['b'] = e
df2['c'] = f

In [ ]:
df2

In [ ]:
df.append(df2)